# In this notebook we will evaluate our portfolio's makeup

### Installation of libraries (consider conda install for working env first)

In [1]:
# !pip install altair
# !pip install pandas_ta
# !pip install requests_html
# !pip install requests
# pip install scikit-plot
# !pip install --user tensorflow
# pip install --upgrade pip
# !pip install yahoo_fin --upgrade --no-cache-dir
# !pip install yfinance --upgrade --no-cache-dir
# !pip install --user pycaret
# pip install scikit-learn==0.23.2
# pip install benzinga
# !pip install --user yahooquery
# !pip install pycaret
# !pip install mplfinance
# !pip install requests_cache
# !pip install certifi
# !pip install polygon-api-client~=1.0.0b
# !pip install --upgrade yfinance
# !pip install yfinance==0.2.24
# !pip install yahoofinancials
# !pip install altair_saver
# !pip install vl-convert-python
# !pip install -U altair_viewer
# !pip install altair_viewer==4.17.0
# !pip install pdfkit

### Importing relevant libraries

In [2]:
from polygon import RESTClient
import getpass
from datetime import date
 
# Returns the current local date
today = date.today()
dateOfMonth = today.day
monthNumber = today.month
yearNumber = today.year

import ftplib
import io
import pandas as pd
import pandas_ta as ta
import requests
import requests_html
import numpy as np
import importlib

# from pycaret.classification import *
# from pycaret.regression import *

import yfinance as yf
importlib.reload(yf)
from yahoo_fin.stock_info import get_data, get_top_crypto, get_analysts_info
import yahoo_fin.stock_info as si
importlib.reload(si)
import yahoo_fin.options as ops
from yahoo_fin.stock_info import *

# Dow: tickers_dow()
# Nasdaq: tickers_nasdaq()
# S&P500: tickers_sp500()
# Others: tickers_other()

# import tensorflow as tf
import altair as alt

# from benzinga import financial_data
# from benzinga import news_data

import mplfinance as mpf

import requests_cache

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D 

import metricFxns
from metricFxns import *
import vizFxns
from vizFxns import *

from sklearn import metrics
from sklearn import datasets

from yahooquery import Ticker

import ssl
import certifi
import urllib

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

from yahoo_fin import stock_info as si

# Assuming df is your DataFrame
pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

# Define custom theme as a dictionary
my_custom_theme = {
    'config': {
        'axis': {
            'labelFont': 'Playfair Display',
            'titleFont': 'Playfair Display',
            'labelFontSize': 12,  # Adjust font size as needed
            'titleFontSize': 14  # Adjust font size as needed
        },
        'text': {
            'align': 'left',  # Align text to the left
            'font': 'Playfair Display',
            'fontSize': 12  # Adjust font size as needed
        },
        'grid': {
            'color': 'lightgray',  # Lighter vertical gridlines
            'strokeWidth': 0.5
        }
    }
}

# Register custom theme
alt.themes.register('my_custom_theme', my_custom_theme)
alt.themes.enable('my_custom_theme')

ThemeRegistry.enable('my_custom_theme')

In [3]:
currentDate = str(monthNumber) + "/" + str(dateOfMonth) + "/" + str(yearNumber)
yearPrior = str(monthNumber) + "/" + str(dateOfMonth+1) + "/" + str(yearNumber-1)
yearPrior

'5/26/2023'

In [4]:
roth = pd.read_csv("D:\Roth.csv")
indv = pd.read_csv("D:\Indv.csv")

roth.columns = list(roth.iloc[1].values)
roth = roth.iloc[2:]
roth = roth.iloc[:-1]

indv.columns = list(indv.iloc[1].values)
indv = indv.iloc[2:]
indv = indv.iloc[:-1]

indv['Market Value'] = indv['Market Value'].str.replace('$','').str.replace(',','').astype(float)
indv['% Of Account'] = indv['% Of Account'].str.replace('%','').astype(float)

roth['Market Value'] = roth['Market Value'].str.replace('$','').str.replace(',','').astype(float)
roth['% Of Account'] = roth['% Of Account'].str.replace('%','').astype(float)

# Define the new row to be added
new_row = {'Symbol': 'Cash & Cash Investments','Description': 'Cash and Money Market', 'Security Type': 'Cash and Money Market', 'Market Value': 25000}
 
# Use the loc method to add the new row to the DataFrame
indv.loc[len(indv)] = new_row

selected_account = indv

#cash or no cash included
# display(selected_account)
# selected_account = selected_account[selected_account['Symbol']!='Cash & Cash Investments']
selected_account = selected_account[selected_account['Security Type']!='Cash and Money Market']

totalValue = selected_account['Market Value'].sum()

selected_account['% Of Account'] = selected_account['Market Value']/totalValue

# indv

C:\Users\npess\AppData\Local\Temp/ipykernel_11564/3145164953.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  indv['Market Value'] = indv['Market Value'].str.replace('$','').str.replace(',','').astype(float)
C:\Users\npess\AppData\Local\Temp/ipykernel_11564/3145164953.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  roth['Market Value'] = roth['Market Value'].str.replace('$','').str.replace(',','').astype(float)
C:\Users\npess\AppData\Local\Temp/ipykernel_11564/3145164953.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [5]:
selected_account = selected_account[['Symbol','% Of Account','Security Type']]

selected_account = selected_account.groupby(['Symbol','Security Type']).sum().reset_index()
selected_account

,Symbol,Security Type,% Of Account
0,A,Equity,0.005378
1,ADBE,Equity,0.050982
2,ADSK,Equity,0.023014
3,AEP,Equity,0.003176
4,ALB,Equity,0.004555
...,...,...,...
60,V,Equity,0.009799
61,VEEV,Equity,0.007279
62,WMT,Equity,0.011670
63,XOM,Equity,0.008098


In [6]:
sector_etfs = ['XLE','XLV','XLU','XLI','GDX','XLK','XLY','XLB','XOP','XHB','XME','XTL']
current_portfolio = list(selected_account['Symbol'].values)

In [7]:
infoDfs = []

for stock in current_portfolio:#,'AMD','INTC','META','AAPL','AMZN','GOOGL','MSFT','TSLA','KO','NVDA']:
    try:
        stock = stock.replace('/','-')
    except:
        pass

    if stock == 'Cash & Cash Investments':
        pass
    else:
        # try:
        ticker = yf.Ticker(stock.lower())

        # # Get quarterly earnings for the last 5 years
        # quarterly_earnings = ticker.quarterly_earnings

        # # Get quarterly cash flow for the last 5 years
        # quarterly_cashflow = ticker.quarterly_cashflow

        # # Filter the data to keep only the last 20 quarters (5 years)
        # quarterly_earnings = quarterly_earnings.tail(20)
        # quarterly_cashflow = quarterly_cashflow.tail(20)

        # print(quarterly_earnings)
        # print(quarterly_cashflow)

        infoDf = pd.DataFrame.from_dict(ticker.info, orient='index').T
        infoDfs.append(infoDf)
        # except:
        #     pass

# infoDf = pd.DataFrame.from_dict(yf.Ticker("aapl").info, orient='index').T
infoDf = pd.concat(infoDfs)
infoDf.head()


,address1,city,state,zip,country,phone,fax,website,industry,industryKey,...,totalAssets,navPrice,category,ytdReturn,beta3Year,fundFamily,fundInceptionDate,legalType,threeYearAverageReturn,fiveYearAverageReturn
0,5301 Stevens Creek Boulevard,Santa Clara,CA,95051,United States,800 227 9770,866 497 1134,https://www.agilent.com,Diagnostics & Research,diagnostics-research,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,345 Park Avenue,San Jose,CA,95110-2704,United States,408 536 6000,408 537 6000,https://www.adobe.com,Software - Infrastructure,software-infrastructure,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,One Market Street,San Francisco,CA,94105,United States,415 507 5000,NaN,https://www.autodesk.com,Software - Application,software-application,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1 Riverside Plaza,Columbus,OH,43215-2373,United States,614 716 1000,NaN,https://www.aep.com,Utilities - Regulated Electric,utilities-regulated-electric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4250 Congress Street,Charlotte,NC,28209,United States,980 299 5700,980 299 5512,https://www.albemarle.com,Specialty Chemicals,specialty-chemicals,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
infoDf.columns

Index(['address1', 'city', 'state', 'zip', 'country', 'phone', 'fax',
       'website', 'industry', 'industryKey',
       ...
       'totalAssets', 'navPrice', 'category', 'ytdReturn', 'beta3Year',
       'fundFamily', 'fundInceptionDate', 'legalType',
       'threeYearAverageReturn', 'fiveYearAverageReturn'],
      dtype='object', length=146)

In [9]:
selected_account['Symbol']=selected_account['Symbol'].replace('BRK/B','BRK-B')
infoDfFull = infoDf.merge(selected_account,how='outer',left_on='symbol',right_on='Symbol')
infoDfFull

,address1,city,state,zip,country,phone,fax,website,industry,industryKey,...,ytdReturn,beta3Year,fundFamily,fundInceptionDate,legalType,threeYearAverageReturn,fiveYearAverageReturn,Symbol,Security Type,% Of Account
0,5301 Stevens Creek Boulevard,Santa Clara,CA,95051,United States,800 227 9770,866 497 1134,https://www.agilent.com,Diagnostics & Research,diagnostics-research,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,Equity,0.005378
1,345 Park Avenue,San Jose,CA,95110-2704,United States,408 536 6000,408 537 6000,https://www.adobe.com,Software - Infrastructure,software-infrastructure,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ADBE,Equity,0.050982
2,One Market Street,San Francisco,CA,94105,United States,415 507 5000,NaN,https://www.autodesk.com,Software - Application,software-application,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ADSK,Equity,0.023014
3,1 Riverside Plaza,Columbus,OH,43215-2373,United States,614 716 1000,NaN,https://www.aep.com,Utilities - Regulated Electric,utilities-regulated-electric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AEP,Equity,0.003176
4,4250 Congress Street,Charlotte,NC,28209,United States,980 299 5700,980 299 5512,https://www.albemarle.com,Specialty Chemicals,specialty-chemicals,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALB,Equity,0.004555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,PO Box 8999,San Francisco,CA,94128-8999,United States,650 432 3200,NaN,https://www.visa.com,Credit Services,credit-services,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V,Equity,0.009799
61,4280 Hacienda Drive,Pleasanton,CA,94588,United States,925 452 6500,925 452 6504,https://www.veeva.com,Health Information Services,health-information-services,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VEEV,Equity,0.007279
62,702 South West 8th Street,Bentonville,AR,72716,United States,479 273 4000,NaN,https://corporate.walmart.com,Discount Stores,discount-stores,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WMT,Equity,0.011670
63,22777 Springwoods Village Parkway,Spring,TX,77389-1425,United States,972-940-6000,NaN,https://corporate.exxonmobil.com,Oil & Gas Integrated,oil-gas-integrated,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XOM,Equity,0.008098


In [10]:
infoDfFull['sector'] = np.where(
    infoDfFull['sector'].isna(),
    infoDfFull['Security Type'],
    infoDfFull['sector']
)

infoDfFull['industry'] = np.where(
    infoDfFull['industry'].isna(),
    infoDfFull['Security Type'],
    infoDfFull['industry']
)

infoDfFull['longName'] = np.where(
    infoDfFull['longName'].isna(),
    infoDfFull['Security Type'],
    infoDfFull['longName']
)

infoDfFull

,address1,city,state,zip,country,phone,fax,website,industry,industryKey,...,ytdReturn,beta3Year,fundFamily,fundInceptionDate,legalType,threeYearAverageReturn,fiveYearAverageReturn,Symbol,Security Type,% Of Account
0,5301 Stevens Creek Boulevard,Santa Clara,CA,95051,United States,800 227 9770,866 497 1134,https://www.agilent.com,Diagnostics & Research,diagnostics-research,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,Equity,0.005378
1,345 Park Avenue,San Jose,CA,95110-2704,United States,408 536 6000,408 537 6000,https://www.adobe.com,Software - Infrastructure,software-infrastructure,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ADBE,Equity,0.050982
2,One Market Street,San Francisco,CA,94105,United States,415 507 5000,NaN,https://www.autodesk.com,Software - Application,software-application,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ADSK,Equity,0.023014
3,1 Riverside Plaza,Columbus,OH,43215-2373,United States,614 716 1000,NaN,https://www.aep.com,Utilities - Regulated Electric,utilities-regulated-electric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AEP,Equity,0.003176
4,4250 Congress Street,Charlotte,NC,28209,United States,980 299 5700,980 299 5512,https://www.albemarle.com,Specialty Chemicals,specialty-chemicals,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALB,Equity,0.004555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,PO Box 8999,San Francisco,CA,94128-8999,United States,650 432 3200,NaN,https://www.visa.com,Credit Services,credit-services,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V,Equity,0.009799
61,4280 Hacienda Drive,Pleasanton,CA,94588,United States,925 452 6500,925 452 6504,https://www.veeva.com,Health Information Services,health-information-services,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VEEV,Equity,0.007279
62,702 South West 8th Street,Bentonville,AR,72716,United States,479 273 4000,NaN,https://corporate.walmart.com,Discount Stores,discount-stores,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WMT,Equity,0.011670
63,22777 Springwoods Village Parkway,Spring,TX,77389-1425,United States,972-940-6000,NaN,https://corporate.exxonmobil.com,Oil & Gas Integrated,oil-gas-integrated,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XOM,Equity,0.008098


In [11]:
sectorDf = infoDfFull.groupby(['sector']).sum().reset_index()
sp500weighting = {'Basic Materials':.024,
                  'Communication Services':.088,
                  'Consumer Cyclical':.102,
                  'Consumer Defensive':.069,
                  'ETFs & Closed End Funds':0,
                  'Energy':.042,
                  'Financial Services':.125,
                  'Healthcare':.137,
                  'Industrials':.082,
                  'Technology':.28,
                  'Utilities':.027}
sectorDf['Target'] = sectorDf['sector'].map(sp500weighting)
display(sectorDf)

,sector,% Of Account,Target
0,Basic Materials,0.004555,0.024
1,Communication Services,0.084306,0.088
2,Consumer Cyclical,0.100082,0.102
3,Consumer Defensive,0.021468,0.069
4,ETFs & Closed End Funds,0.011686,0.000
5,Energy,0.043204,0.042
6,Financial Services,0.096627,0.125
7,Healthcare,0.094044,0.137
8,Industrials,0.063811,0.082
9,Real Estate,0.003429,NaN


In [49]:
# Disable any previously enabled themes
alt.themes.enable("default")
# Define color scale for sector
color_scale_map = {'Basic Materials':'#997B66',
                   'Communication Services':'#E8AC65',
                   'Consumer Cyclical':'#FFCB69',
                   'Consumer Defensive':'#E2D5CD',
                   'ETFs & Closed End Funds':'#9B8E85',
                   'Energy':'#797D62',
                   'Financial Services':'#9B9B7A',
                   'Healthcare':'#D9AE94',
                   'Industrials':'#B58463',
                   'Real Estate':'#F1DCA7',
                   'Technology':'#D08C60',
                   'Utilities':'#BAA587'}

# Define color encoding for sector with the mapped colors
color_encoding = alt.Color('sector', scale=alt.Scale(domain=list(color_scale_map.keys()), range=list(color_scale_map.values())))

# Your original chart code
base = alt.Chart(sectorDf).encode(
    x=alt.X('% Of Account', axis=alt.Axis(format='%',title=None))
)

barChart = base.mark_bar().encode(
    x=alt.X('% Of Account', axis=alt.Axis(format='%',title=None)),
    y=alt.Y('sector', axis=alt.Axis(title=None)),  # No y-axis title
    color=color_encoding
).properties(
    height=300,
    width=600
)

tickChart = base.mark_tick(size=15, color='black', thickness=3).encode(
    x=alt.X('Target', axis=alt.Axis(format='%',title=None)),
    y=alt.Y('sector', axis=alt.Axis(title=None))  # No y-axis title
).properties(
    height=300,
    width=600
)

# Layer the charts
layered_chart = alt.layer(barChart, tickChart, data=sectorDf)

# Apply axis and text configurations directly
layered_chart = layered_chart.configure_axis(
    labelFont="Playfair Display",
    titleFont="Playfair Display",
    labelFontSize=12,
    titleFontSize=14,
    labelColor="#696969",  # Font color
    titleColor="#696969"   # Font color
).configure_text(
    font='Playfair Display',
    fontSize=12,
    align='left',
    color="#696969"  # Font color
)

# Configure y-axis specifically
layered_chart = layered_chart.configure_axisY(
    labelPadding=150,  # Padding between labels and axis
    labelAlign='left'  # Left align y-axis labels
)

# Set chart title properties directly
layered_chart = layered_chart.properties(
    title=alt.TitleParams(
        text="Sector Allocation with S&P Benchmark",
        fontSize=24,
        font = 'Playfair Display',
        dy=-20,
        color="#696969",  # Font color
        anchor="start"  # Left-aligned title
    )
)

# Apply legend configurations directly
layered_chart = layered_chart.configure_legend(
    titleFont='Playfair Display',
    titleFontSize=12,
    labelFont='Playfair Display',
    labelFontSize=10,
    titleColor="#696969",  # Font color
    labelColor="#696969"   # Font color
)

layered_chart


alt.LayerChart(...)

In [50]:

import os
import altair_saver
from altair_saver import save
importlib.reload(altair_saver)

# Specify the directory path
directory = "C:/Financial Outputs/Overviews"

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Specify the path and filename for the PNG image
png_file = os.path.join(directory, "chart.png")

# Save the Altair chart as a PNG image
layered_chart.save(png_file, engine="vl-convert")


In [51]:
# barChart = alt.Chart(infoDfFull).mark_bar().encode(
#     x=alt.X('% Of Account',axis=alt.Axis(format='%')),
#     y=alt.Y('longName'),
#     color=alt.Color('sector', scale=alt.Scale(scheme = 'set3'))
# ).properties(
#     height=1200,
#     width=600
# )

infoDfFull = infoDfFull.sort_values(by='% Of Account',ascending=False)

base = alt.Chart(infoDfFull.head(10)).encode(
    x=alt.X('% Of Account', axis=alt.Axis(format='%'), title=None)
)

barChart = base.mark_bar().encode(
    x=alt.X('% Of Account', axis=alt.Axis(format='%'), title=None),
    y=alt.Y('longName', axis=alt.Axis(title=None),sort='-x'),  # No y-axis title
    color=color_encoding
).properties(
    height=300,  # Adjusted height
    width=600
)

# Apply axis and text configurations directly
barChart = barChart.configure_axis(
    labelFont="Playfair Display",
    titleFont="Playfair Display",
    labelFontSize=12,
    titleFontSize=14,
    labelColor="#696969",  # Font color
    titleColor="#696969"   # Font color
).configure_text(
    font='Playfair Display',
    fontSize=12,
    align='left',
    color="#696969"  # Font color
)

# Configure y-axis specifically
barChart = barChart.configure_axisY(
    labelPadding=150,  # Padding between labels and axis
    labelAlign='left'  # Left align y-axis labels
)

# Set chart title properties directly
barChart = barChart.properties(
    title=alt.TitleParams(
        text="Top 10 Stock Allocation",
        fontSize=24,
        font='Playfair Display',
        dy=-20,
        color="#696969",  # Font color
        anchor="start"  # Left-aligned title
    )
)

# Apply legend configurations directly
barChart = barChart.configure_legend(
    titleFont='Playfair Display',
    titleFontSize=12,
    labelFont='Playfair Display',
    labelFontSize=10,
    titleColor="#696969",  # Font color
    labelColor="#696969"   # Font color
)

barChart

# Specify the path and filename for the PNG image
stock_allocation_png_file = os.path.join(directory, "stockAllocationChart.png")

# Save the Altair chart as a PNG image
barChart.save(stock_allocation_png_file, engine="vl-convert")

In [52]:
list(infoDfFull.columns)
nan_percentage = (infoDfFull.isna().sum(axis=0) / len(infoDfFull)) * 100
nan_percentage_df = pd.DataFrame(nan_percentage, columns=['Percentage of NaN'])
nan_percentage_df['field'] = nan_percentage_df.index
# nan_percentage_df.set_index('field', inplace=True)

In [72]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.utils import ImageReader
from PIL import Image
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics


# Create PDF canvas
pdf_file = os.path.join(directory, "Portfolio Overview.pdf")
c = canvas.Canvas(pdf_file, pagesize=letter)

#Sector allocation

# Get the original dimensions of the image
img = Image.open(png_file)
width, height = img.size

# Calculate the aspect ratio
aspect_ratio = width / height

# Set the desired width and calculate the corresponding height to maintain aspect ratio
desired_width = 500
desired_height = desired_width / aspect_ratio

# Draw the image on the PDF canvas
c.drawImage(png_file, x=50, y=500, width=desired_width, height=desired_height)

#Stock allocation
# Get the original dimensions of the image
img = Image.open(png_file)
width, height = img.size

# Calculate the aspect ratio
aspect_ratio = width / height

# Set the desired width and calculate the corresponding height to maintain aspect ratio
desired_width = 500
desired_height = desired_width / aspect_ratio

# Draw the image on the PDF canvas
c.drawImage(stock_allocation_png_file, x=50, y=275, width=desired_width, height=desired_height)

# Create a new page
c.showPage()

#Forecast
# Add text to the PDF
# Set the font to Playfair Display
# Register the Playfair Display font
# pdfmetrics.registerFont(TTFont('PlayfairDisplay', 'PlayfairDisplay-Regular.ttf'))
# c.setFont("Helvetica", 12)
# c.setFillColorRGB(0.41, 0.41, 0.41)
# text = "Adobe 12 month forecast"
# c.drawString(5, 70, text)

# Get the original dimensions of the image
forecast_png = os.path.join(directory, "forecast_chart.png")
img = Image.open(forecast_png)
width, height = img.size

# Calculate the aspect ratio
aspect_ratio = width / height

# Set the desired width and calculate the corresponding height to maintain aspect ratio
desired_width = 600
desired_height = desired_width / aspect_ratio

# Draw the image on the PDF canvas
c.drawImage(forecast_png, x=-5, y=70, width=desired_width, height=desired_height)

# Save the PDF
c.showPage()
c.save()